# ایزور اے آئی ایجنٹ سروس اور سیمینٹک کرنل کا استعمال کرتے ہوئے RAG

یہ کوڈ نمونہ دکھاتا ہے کہ کس طرح `Azure AI Agent Service` اور `Semantic Kernel` کا استعمال کرتے ہوئے ریٹریول-آگمینٹڈ جنریشن (RAG) کے لیے ایک ایزور اے آئی ایجنٹ بنایا اور منظم کیا جا سکتا ہے۔ یہ ایجنٹ صارف کے سوالات کو حاصل کردہ سیاق و سباق کی بنیاد پر پروسیس کرتا ہے اور اس کے مطابق درست جوابات فراہم کرتا ہے۔


SQLite ورژن کی درستگی  
اگر آپ کو یہ خرابی پیش آئے:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

اپنے نوٹ بک کے آغاز میں اس کوڈ بلاک کو غیر تبصرہ کریں:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### پیکجز درآمد کرنا  
مندرجہ ذیل کوڈ ضروری پیکجز کو درآمد کرتا ہے:  


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# سیمینٹک کرنل اور ایژر اے آئی ایجنٹ سروس کے ساتھ ریٹریول-اگمینٹڈ جنریشن

یہ نمونہ دکھاتا ہے کہ **ایژر اے آئی ایجنٹ سروس** کا استعمال کیسے کیا جائے تاکہ ایک زبان کے ماڈل کو اپلوڈ شدہ دستاویز سے حاصل کردہ ڈومین-اسپیسفک مواد کے ساتھ ملا کر **ریٹریول-اگمینٹڈ جنریشن (RAG)** انجام دی جا سکے۔

### یہ کیسے کام کرتا ہے

1. **دستاویز اپلوڈ**: ایک مارک ڈاؤن فائل (document.md) جس میں معلومات (کنٹوسو کی سفری انشورنس پالیسی) شامل ہیں، ایجنٹ سروس پر اپلوڈ کی جاتی ہے۔

2. **ویکٹر اسٹور کی تخلیق**: دستاویز کو ویکٹر اسٹور میں انڈیکس کیا جاتا ہے تاکہ اس کے مواد پر سیمینٹک سرچ ممکن ہو سکے۔

3. **ایجنٹ کی تشکیل**: `gpt-4o` ماڈل کے ساتھ ایک ایجنٹ کو درج ذیل سخت ہدایات کے ساتھ تشکیل دیا جاتا ہے:
   - صرف دستاویز سے حاصل کردہ مواد کی بنیاد پر سوالات کے جوابات دے۔
   - اگر سوال دائرہ کار سے باہر ہو تو جواب دینے سے انکار کرے۔

4. **فائل سرچ ٹول کا انضمام**: `FileSearchTool` کو ایجنٹ کے ساتھ رجسٹر کیا جاتا ہے، جو ماڈل کو انڈیکس شدہ دستاویز سے متعلقہ ٹکڑوں کو تلاش کرنے اور انفرینس کے دوران بازیافت کرنے کے قابل بناتا ہے۔

5. **صارف کے ساتھ تعامل**: صارف سوالات پوچھ سکتے ہیں۔ اگر دستاویز میں متعلقہ معلومات موجود ہوں، تو ایجنٹ ایک مستند جواب تیار کرتا ہے۔  
   اگر نہیں، تو ایجنٹ واضح طور پر جواب دیتا ہے کہ دستاویز میں کافی معلومات موجود نہیں ہیں۔


### مرکزی فنکشن

یہ سیکشن مرکزی فنکشن کی وضاحت کرتا ہے، جو پروگرام کے آغاز کا نقطہ ہے۔

#### کوڈ کی وضاحت

مرکزی فنکشن درج ذیل کام انجام دیتا ہے:

1. ابتدائی متغیرات کو ترتیب دیتا ہے۔
2. صارف کی ان پٹ کو پروسیس کرتا ہے۔
3. نتائج کو آؤٹ پٹ کرتا ہے۔

#### مثال

ذیل میں ایک سادہ مثال دی گئی ہے جو مرکزی فنکشن کے بنیادی استعمال کو ظاہر کرتی ہے:

```python
def main():
    # صارف کو خوش آمدید پیغام دکھائیں
    print("پروگرام میں خوش آمدید!")
    
    # صارف سے ان پٹ لیں
    user_input = input("براہ کرم ایک عدد درج کریں: ")
    
    # ان پٹ کو پروسیس کریں
    result = process_input(user_input)
    
    # نتیجہ دکھائیں
    print(f"آپ کا نتیجہ: {result}")
```

#### نوٹ

- یقینی بنائیں کہ تمام ضروری ماڈیولز کو امپورٹ کیا گیا ہے۔
- `process_input` فنکشن کو پہلے سے ڈیفائن کریں تاکہ یہ صحیح طریقے سے کام کرے۔


یقینی بنائیں کہ Azure CLI کا استعمال کرتے ہوئے پہلے `az login` چلائیں تاکہ `DefaultAzureCredential` استعمال کرتے وقت مناسب تصدیقی سیاق و سباق فراہم کیا جا سکے۔ Azure AI Agent Service API کیز استعمال نہیں کرتا۔


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

ضروری اصول:  
1. اس بات کو یقینی بنائیں کہ آپ کے ترجمے میں '''مارک ڈاؤن یا کسی اور ٹیگ کا اضافہ نہ ہو۔  
2. ترجمہ بہت زیادہ لفظی نہ لگے۔  
3. تبصرے بھی ترجمہ کریں۔  
4. یہ فائل مارک ڈاؤن فارمیٹ میں لکھی گئی ہے - اسے XML یا HTML نہ سمجھیں۔  
5. درج ذیل کو ترجمہ نہ کریں:  
   - [!NOTE], [!WARNING], [!TIP], [!IMPORTANT], [!CAUTION]  
   - ویری ایبل کے نام، فنکشن کے نام، کلاس کے نام  
   - پلیس ہولڈرز جیسے @@INLINE_CODE_x@@ یا @@CODE_BLOCK_x@@  
   - URLs یا راستے  
6. تمام اصل مارک ڈاؤن فارمیٹنگ کو برقرار رکھیں۔  
7. صرف ترجمہ شدہ مواد واپس کریں، اضافی ٹیگز یا مارک اپ کے بغیر۔  

آپ کو نتیجہ کچھ اس طرح نظر آنا چاہیے:  

```
# User: 'Can you explain Contoso's travel insurance coverage?'
# Agent: Contoso's travel insurance coverage includes protection for medical emergencies, trip cancellations, and lost baggage【4:0†document.md】.
# User: 'What is a Neural Network?'
# Agent: I'm sorry, but the uploaded document does not contain the necessary information to answer that question.

Cleaned up agent, thread, file, and vector store.
```  



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے پوری کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا خامیاں ہو سکتی ہیں۔ اصل دستاویز، جو اس کی اصل زبان میں ہے، کو مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ تجویز کیا جاتا ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے لیے ہم ذمہ دار نہیں ہیں۔
